In [88]:
import gzip
import json

In [89]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        l = str(json.loads(l))
        yield eval(l)

In [90]:
data_modcloth = []
for l in parse("modcloth_final_data.json.gz"):
    
    data_modcloth.append(l)

In [91]:
data_renttherunway = []
for l in parse("renttherunway_final_data.json.gz"):
    
    data_renttherunway.append(l)

In [93]:
data_renttherunway[0]

dict